In [11]:
stream = [0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1]
iv = bytes.fromhex('cd2832f408d1d973be28b66b133a0b5f')
encrypted_flag = bytes.fromhex('1e3c272c4d9693580659218739e9adace2c5daf98062cf892cf6a9d0fc465671f8cd70a139b384836637c131217643c1')

In [110]:
class LFSR:
    def __init__(self, key, taps):
        d = max(taps)
        assert len(key) == d, "Error: key of wrong size."
        self._s = key
        self._t = [d - t for t in taps]

    def _sum(self, L):
        s = 0
        for x in L:
            s ^^= x
        return s

    def _clock(self):
        b = self._s[0]
        self._s = self._s[1:] + [self._sum(self._s[p] for p in self._t)]
        return b

    def getbit(self):
        return self._clock()

In [123]:
tapses = [
    [19, 18, 17, 14],
    [27, 26, 25, 22],
    [23, 22, 20, 18],
]

In [51]:
taps = [27, 26, 25, 22]
sz = max(taps)
key = getrandbits(sz)
key = [int(i) for i in list(f"{{:0{sz}b}}".format(key))]
print(key)

[1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0]


In [79]:
F.<x> = PolynomialRing(GF(2))
P = sum(pow(x,sz-r) for r in taps + [0])
C = companion_matrix(P, format='bottom')
print(P)
print(C[-1])

x^27 + x^5 + x^2 + x + 1
(1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [118]:
g = 50

rng = LFSR(key, taps)
for _ in range(g):
    rng.getbit()

v = [rng.getbit() for _ in range(sz)]
u = C^g * vector(key)

assert u == vector(v)

In [119]:
b = vector(v)
A = C^g
x = A^-1 * b

assert x == vector(key)

In [125]:
import itertools
k = 2
key_len = max(tapses[k])

for b in range(key_len):
    print(b)
    for c in itertools.combinations(range(key_len), b):
        key_condidate = [1 - stream[i] if i in c else stream[i] for i in range(key_len)]
        lfsr = LFSR(key_condidate, tapses[k])
        matches = sum(x == lfsr.getbit() for x in stream)
        if matches >= 180:
            print(key_condidate)

0
1
2
3
4
5


KeyboardInterrupt: 